# Combine DLC positions with my tracklets to impute missing values

In [1]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import pandas as pd
import numpy as np
import napari
from DLC_for_WBFM.utils.visualization.napari_from_config import dlc_to_napari_tracks
from scipy.spatial import distance
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
# project_fname = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
project_fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
project_cfg = load_config(project_fname)
segment_fname = project_cfg['subfolder_configs']['segmentation']
project_dir = Path(project_fname).parent

tracklet_fname = os.path.join('2-training_data', 'all_tracklets.h5')
dlc_fname = os.path.join('3-tracking', 'full_3d_tracks.h5')

with safe_cd(project_dir):
    segment_cfg = load_config(segment_fname)
    df_tracklet = pd.read_hdf(tracklet_fname)
    dlc_tracks = pd.read_hdf(dlc_fname)

In [3]:
dlc_tracks.replace(0, np.NaN, inplace=True)

In [4]:
red_data_fname = project_cfg['preprocessed_red']
red_data = zarr.open(red_data_fname)

In [5]:
napari_tracklets = dlc_to_napari_tracks(df_tracklet, 0.0)

In [7]:
# Visualize everything
viewer = napari.view_image(red_data)
viewer.add_tracks(napari_tracklets)

<Tracks layer 'napari_tracklets' at 0x161737e2dc8>

Unable to load numpy_formathandler accelerator from OpenGL_accelerate


In [6]:
# For one DLC track, look at the distribution of tracklet distances
def calc_dlc_to_tracklet_distances(dlc_tracks, df_tracklet, dlc_name, all_covering_ind, min_overlap=5, min_dlc_confidence=0.6):
    """For one DLC neuron, calculate distances between that track and all tracklets"""
    coords = ['z', 'x', 'y']

    this_dlc = dlc_tracks[dlc_name][coords]
    # Remove low confidence points
#     low_conf = this_dlc['likelihood'] < min_dlc_confidence
#     this_dlc.loc['x', low_conf].replace()

    all_tracklet_names = list(df_tracklet.columns.levels[0])
    all_dist = []
    for i, name in enumerate(tqdm(all_tracklet_names, leave=False)):
        # Check for already belonging to another track
        if i not in all_covering_ind:
            this_diff = df_tracklet[name][coords] - this_dlc
            
            # Check for enough common data points
            num_common_pts = this_diff['x'].notnull().sum()
            if num_common_pts >= min_overlap:
                dist = np.linalg.norm(this_diff, axis=1)
            else:
                dist = np.inf
        else:
            dist = np.inf
        
        all_dist.append(dist)
    
    return all_dist


def calc_covering_from_distances(all_dist, df_tracklet, all_covering_ind, d_max=5):
    """Given distances between a dlc track and all tracklets, make a time-unique covering from the tracklets"""
    all_meds = list(map(np.nanmedian, all_dist))
    ind = np.argsort(all_meds)
    all_tracklet_names = list(df_tracklet.columns.levels[0])

    covering_ind = []
    covering_time_points = []
    t = df_tracklet.index
    these_dist = np.zeros_like(t, dtype=float)
    
    for i in ind:
        # Check if this was used before
        for old_coverings in all_covering_ind:
            if i in old_coverings:
                continue
        # Check distance
        if all_meds[i] > d_max:
            break
        # Check time overlap
        name = all_tracklet_names[i]
        is_nan = df_tracklet[name]['x'].isnull()
        new_t = list(t[~is_nan])
        if any([t in covering_time_points for t in new_t]):
            continue

        # Save
        new_t = np.array(new_t)
        covering_time_points.extend(new_t)
        covering_ind.append(i)
        these_dist[new_t] = all_dist[i][new_t]
#         these_dist.append(all_dist[i])
    print(f"Covering of length {len(covering_time_points)} made from {len(covering_ind)} tracklets")
        
    return covering_time_points, covering_ind, these_dist

In [7]:
all_dlc_names = list(dlc_tracks.columns.levels[0])
dlc_name = all_dlc_names[0]

all_covering_dist = []
all_covering_time_points = []
all_covering_ind = []
for dlc_name in tqdm(all_dlc_names):
    dist = calc_dlc_to_tracklet_distances(dlc_tracks, df_tracklet, dlc_name, all_covering_ind)
    covering_time_points, covering_ind, these_dist = calc_covering_from_distances(dist, df_tracklet, all_covering_ind, d_max=5)
    
    all_covering_dist.append(these_dist)
    all_covering_time_points.append(covering_time_points)
    all_covering_ind.append(covering_ind)
    break

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 785 made from 8 tracklets


In [72]:
# Distances
# %matplotlib widget
# plt.plot(all_covering_dist[0])
# np.min(all_covering_dist[0])

In [8]:
# Visualize coverings DLC + tracklets
i = 0
t_tracklet_covering = all_covering_time_points[i]
# ind_covering = all_covering_ind[i]
df_dlc = dlc_tracks[all_dlc_names[i]]
is_nan = df_dlc['x'].isnull()
t = df_dlc.index
t_dlc_covering = list(t[~is_nan])

# Process into: both, one, or neither
t_both = []
t_tracklet_only = []
t_dlc_only = []
t_gap = []
set_dlc, set_tracklet = set(t_dlc_covering), set(t_tracklet_covering)

for i in t:
    in_dlc = i in set_dlc
    in_tracklet = i in set_tracklet
    if in_dlc and in_tracklet:
        t_both.append(i)
    elif in_dlc:
        t_dlc_only.append(i)
    elif in_tracklet:
        t_tracklet_only.append(i)
    else:
        t_gap.append(i)
        

In [30]:
# %matplotlib widget
# plt.plot(t_both, 1.3*np.ones_like(t_both), 'o', label=f'Covered by both: {len(t_both)}')
# plt.plot(t_dlc_only, 1.2*np.ones_like(t_dlc_only), 'o', label=f'Points covered by only DLC: {len(t_dlc_only)}')
# plt.plot(t_tracklet_only, 1.1*np.ones_like(t_tracklet_only), 'o', label=f'Points covered by only tracklets: {len(t_tracklet_only)}')
# plt.plot(t_gap, np.ones_like(t_gap), 'o', label=f'Gaps: {len(t_gap)}')
# plt.legend()

In [29]:
# %matplotlib widget
# plt.scatter(t_both, 1.3*np.ones_like(t_both), label=f'Covered by both: {len(t_both)}')
# plt.scatter(t_dlc_only, 1.2*np.ones_like(t_dlc_only), label=f'Points covered by only DLC: {len(t_dlc_only)}')
# plt.scatter(t_tracklet_only, 1.1*np.ones_like(t_tracklet_only), label=f'Points covered by only tracklets: {len(t_tracklet_only)}')
# plt.scatter(t_gap, np.ones_like(t_gap), label=f'Gaps: {len(t_gap)}')
# plt.legend()

In [26]:
these_tracklets = all_covering_ind[0]
tracklet_names = (df_tracklet.columns.levels[0])
these_tracklet_names = [tracklet_names[i] for i in these_tracklets]

# Combine tracklets
new_df = df_tracklet[these_tracklet_names]
coords = ['z', 'x', 'y']
all_arrs = []
for c in coords:
    all_arrs.append(np.nansum([new_df[name][c] for name in these_tracklet_names], axis=0))
summed_tracklet_array = np.stack(all_arrs, axis=1)

# Morph to DLC format
cols = [[dlc_tracks.columns.levels[0][0]], coords]
cols = pd.MultiIndex.from_product(cols)
summed_tracklet_df = pd.DataFrame(data=summed_tracklet_array, columns=cols)

# Then cast as napari format
napari_new_tracklets = dlc_to_napari_tracks(summed_tracklet_df, 0.0)

# ALso get original DLC tracks
dlc_name = all_dlc_names[0]
napari_dlc_tracks = dlc_to_napari_tracks(dlc_tracks[[dlc_name]], 0.0)

In [27]:
# Visualize both tracklets on the data

viewer = napari.view_image(red_data)
viewer.add_tracks(napari_new_tracklets)
viewer.add_tracks(napari_dlc_tracks)

<Tracks layer 'napari_dlc_tracks' at 0x2200a5c8848>

In [28]:
summed_tracklet_df

neuron1550                        
              z           x           y
0      8.446753  498.350649  275.957143
1      8.444737  484.360526  289.947368
2      6.431944  480.381944  283.933333
3      8.410738  472.357047  267.971812
4      8.450135  486.397574  247.900270
...         ...         ...         ...
1493   0.000000    0.000000    0.000000
1494   0.000000    0.000000    0.000000
1495   0.000000    0.000000    0.000000
1496   0.000000    0.000000    0.000000
1497   0.000000    0.000000    0.000000

[1498 rows x 3 columns]

In [104]:
# %matplotlib widget
# # # from ipywidgets import interact
# best_tracklets = np.argpartition(all_means, 10)[:10]
# print(best_tracklets)
# plt.hist([all_means[i] for i in best_tracklets])

In [64]:
d = all_dist[0]
np.round(d[~np.isnan(d)], 1)

array([1.1, 0.8, 3.2, 2. , 1.2, 2.4, 0.6, 2.8, 1.7, 1.2, 0.8, 0.9, 2.9,
       0.6, 2.2, 3.7, 0.6, 0.6, 1.6, 2.2])

In [67]:
name = 'neuron1'
this_tracklet = df_tracklet[name][coords]
this_tracklet.iloc[1:20, :]

coords,z,x,y
1,8.444737,484.360526,289.947368
2,6.431944,480.381944,283.933333
3,8.410738,472.357047,267.971812
4,8.450135,486.397574,247.900270
5,8.447958,490.371542,235.891963
6,10.436702,480.283391,217.919861
7,8.451697,452.390339,199.921671
8,8.459060,446.424161,187.893960
9,8.443312,432.313376,177.970701
10,8.426020,406.274235,177.982143


In [68]:
this_dlc.iloc[1:20, :]

coords,z,x,y
1,9,484.520248,290.520874
2,9,482.337982,284.265335
3,9,473.891083,269.164436
4,9,NaN,NaN
5,9,491.375468,235.572243
6,9,482.158783,218.350449
7,9,452.336416,199.883942
8,9,449.116119,187.569519
9,9,430.832611,178.534673
10,9,405.493876,178.703758


In [73]:
len(best_tracklets)

5550